In [1]:
%load_ext sql

In [2]:
#Connecting to a specific database
%sql sqlite:///College.db  

'Connected: @College.db'

In [ ]:
#----------------------------------------------------------------------
#Sub queries in where clause
#----------------------------------------------------------------------
#Exist, all, Any

#1)Colleges within same state

In [3]:
%%sql
select cName, state from College c1
where exists (select * from College c2 
	      where c2.state=c1.state and c1.cName<>c2.cName);

 * sqlite:///College.db
Done.


cName,state
Stanford,CA
Berkeley,CA


In [4]:
#2)Who has hightest enrollment?

In [5]:
%%sql
select cName from College c1
   	where not exists (select * from college c2 where c2.enrollment > c1.enrollment)

 * sqlite:///College.db
Done.


cName
Berkeley


In [18]:
%%sql
select sName, GPA from Student where GPA >=all (select GPA from Student)

 * sqlite:///College.db
(sqlite3.OperationalError) near "all": syntax error
[SQL: select sName, GPA from Student where GPA >=all (select GPA from Student)]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [ ]:
#3) College with highest enrollment 

In [25]:
%%sql
select cName from College s1
          where enrollment > All (select enrollment from College s2 where s2.cName <> s1.cName)

 * sqlite:///College.db
(sqlite3.OperationalError) near "All": syntax error
[SQL: select cName from College s1
          where enrollment > All (select enrollment from College s2 where s2.cName <> s1.cName)]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [26]:
%%sql
select cName from College s1
          where not enrollment <= any (select enrollment from College s2 where s2.cName <> s1.cName)

 * sqlite:///College.db
(sqlite3.OperationalError) near "select": syntax error
[SQL: select cName from College s1
          where not enrollment <= any (select enrollment from College s2 where s2.cName <> s1.cName)]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [ ]:
#4) Students who have applied to Major in CS but not to EE

In [28]:
%%sql
Select sID, sName from Student where sID =  any (select sID from Apply where major = 'cs')
                               and   sID <> all (select sID from Apply where major = 'EE')      

 * sqlite:///College.db
(sqlite3.OperationalError) near "select": syntax error
[SQL: Select sID, sName from Student where sID =  any (select sID from Apply where major = 'cs')
                               and   sID <> all (select sID from Apply where major = 'EE')]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [ ]:
----------------------------------------------------------------------
Sub queries in from and select
----------------------------------------------------------------------

In [32]:
%%sql
select * from (select sID, sName, GPA, GPA*(sizeHS/1000.0) as scaledGPA from Student) G where abs(G.scaledGPA-GPA) > 1.0 

 * sqlite:///College.db
Done.


sID,sName,GPA,scaledGPA
234,Bob,3.6,5.4
345,Craig,3.5,1.75
567,Edward,2.9,5.8
678,Fay,3.8,0.76
876,Irene,3.9,1.56
765,Jay,2.9,4.35
543,Craig,3.4,6.8


In [ ]:
#5) Colleges paired with highest GPA of their applicants

In [42]:
%%sql
select distinct College.cName, state, GPA from College, Apply, Student where College.cName=Apply.cName and 
                                                                                  Apply.sID=Student.sID and
                                                                                GPA >= all  ( select GPA from Student, Apply where Student.sID = Apply.sID and
                                                                                             Apply.cName = College.cName)

 * sqlite:///College.db
(sqlite3.OperationalError) near "all": syntax error
[SQL: select distinct College.cName, state, GPA from College, Apply, Student where College.cName=Apply.cName and 
                                                                                  Apply.sID=Student.sID and
                                                                                GPA >= all  ( select GPA from Student, Apply where Student.sID = Apply.sID and
                                                                                             Apply.cName = College.cName)]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [ ]:
#6)Writing the above qery with sub query in the select clause

In [56]:
%%sql
select cName, state,(select distinct GPA from Apply, Student where College.cName=Apply.cName and 
                     Apply.sID=College.sID and GPA >=all (select GPA from Students where Student.sID=Apply.sID and Apply.cName = College.cName) as GPA 
 from college

 * sqlite:///College.db
(sqlite3.OperationalError) near "all": syntax error
[SQL: select cName, state,(select distinct GPA from Apply, Student where College.cName=Apply.cName and 
                     Apply.sID=College.sID and GPA >=all (select GPA from Students where Student.sID=Apply.sID and Apply.cName = College.cName) as GPA 
 from college]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [ ]:
#------------------------------------------------------------------------
#The Join Family operators 
#------------------------------------------------------------------------

In [58]:
%%sql
select distinct sName, major from student, Apply where Student.sID=Apply.sID

 * sqlite:///College.db
Done.


sName,major
Amy,CS
Amy,EE
Bob,biology
Craig,CS
Craig,EE
Craig,bioengineering
Fay,history
Helen,CS
Irene,CS
Irene,biology


In [60]:
#Equivalent theta join as follows
#--------------------------------

In [62]:
%%sql
select distinct sName, major from student
               inner join
               Apply on Student.sID=Apply.sID;

 * sqlite:///College.db
Done.


sName,major
Amy,CS
Amy,EE
Bob,biology
Craig,CS
Craig,EE
Craig,bioengineering
Fay,history
Helen,CS
Irene,CS
Irene,biology


In [65]:
%%sql
select sName, sID, cName, major from Student left join Apply using(sID)

 * sqlite:///College.db
Done.


sName,sID,cName,major
Amy,123,Berkeley,CS
Amy,123,Cornell,EE
Amy,123,Stanford,CS
Amy,123,Stanford,EE
Bob,234,Berkeley,biology
Craig,345,Cornell,CS
Craig,345,Cornell,EE
Craig,345,Cornell,bioengineering
Craig,345,MIT,bioengineering
Doris,456,None,None


In [ ]:
#Equivalent with out join
#---------------------------------

In [69]:
%%sql
select sName, Student.sID, cName, major
from Student, Apply
where Student.sID=Apply.sID

union

select sName, sID, NULL, NULL 
from Student
where sID not in(select sID from Apply)

 * sqlite:///College.db
Done.


sName,sID,cName,major
Amy,123,Berkeley,CS
Amy,123,Cornell,EE
Amy,123,Stanford,CS
Amy,123,Stanford,EE
Amy,654,None,None
Bob,234,Berkeley,biology
Craig,345,Cornell,CS
Craig,345,Cornell,EE
Craig,345,Cornell,bioengineering
Craig,345,MIT,bioengineering


In [ ]:
#inner join is commutative outer join is not
#inner join is associative outer join is not